In [3]:
import json
#import config
from os import listdir
from une-crawler.crawlerFunctions import config

SyntaxError: invalid syntax (2220987596.py, line 4)

In [2]:
def Transforme(s):
    repetiveis = config.MARC_REPETIVEL

    controfield = {}
    for i in s['controfield']:
        controfield[i['code']] = i['value']

    
    datafield =  {}
    for i in s['datafield']:
        subfield = {}
        for k, v in i['subfield'].items():
            subfield[k] = v.rstrip()
        if i['code'] not in repetiveis:
            datafield[i['code']] = subfield
        else:
            if i['code'] not in datafield.keys():
                datafield[i['code']] = [subfield]
            else:
                datafield[i['code']].append(subfield)
    
    marcJson = {'controfield': controfield, 'datafield': datafield}
    return marcJson

In [58]:
mes = 'mar'
j = listdir(f'out\mcti/{mes}')
for i in j:
    path = f'out\mcti\{mes}/{i}'
    with open(path, encoding="utf-8") as file_json:
        s = json.loads(file_json.read())
    marc = Transforme(s)
    with open(
        f"out/mcti/{i}","w", encoding='utf-8') as jsonfile:
        json.dump(marc, jsonfile, ensure_ascii=False, indent=4)
    print(i)

82055.json


In [ ]:
path = 'out\inpe/01-01-22_31-08-22/7177.json'
with open(path, encoding="utf-8") as file_json:
    s = json.loads(file_json.read())
    file_json.close()
s


In [3]:
def ParserMarc700(v, solr_doc):
    for i in v:
        responsabilty = f'marc700{i.get("e")}'
        if responsabilty not in solr_doc.keys():
            solr_doc[responsabilty] = [i.get('a')]
        else:
            solr_doc[responsabilty].append(i.get('a'))
    return solr_doc
    

In [4]:
def ParserMarc6XX(k, v, solr_doc):
    for i in v:
        
        #Parser Simple
        simple = f'marc{k}a'
        if simple not in solr_doc.keys():
            solr_doc[simple] = [i.get('a')]
        else:
            solr_doc[simple].append(i.get('a'))
        
        #Parser Full
        subfields = list()
        for _, vs in i.items():
            subfields.append(vs)
        subfields = "--".join(subfields)
        full = f'marc{k}completo'
        if full not in solr_doc.keys():
            solr_doc[full] = [subfields]
        else:
            solr_doc[full].append(subfields)
    return solr_doc

In [6]:
def ParserSolr(s, institution):

    solr_doc = {
        'id': int(s.get('controfield').get('001')),
        'institution': institution
    }

    for k, v in s.get('datafield').items():
        if isinstance(v, dict):
            for ks, vs in v.items():
                solr_doc[f'marc{k}{ks}'] = vs
        else:
            #ParserMarc700
            if k == '700':
                solr_doc = ParserMarc700(v, solr_doc)
            #ParserMarc6XX
            elif k[0] == '6':
                solr_doc = ParserMarc6XX(k, v, solr_doc)
            else:
                for subfield in v:
                    for ks, vs in subfield.items():
                        tag = f'marc{k}{ks}'
                        if tag not in solr_doc.keys():
                            solr_doc[tag] = [vs]
                        else:
                            solr_doc[tag].append(vs)
        
    return solr_doc
#solr_doc = ParserSolr(s, 'Goeldi')

In [9]:
solr_doc

{'id': 7177,
 'institution': 'INPE',
 'marc080a': '551.510.42',
 'marc090b': 'Am35L ',
 'marc090a': '551.510.42',
 'marc111a': 'American Meteorological Society Workshop on Meteorology and Environmental Assessment (1975: Boston, MA)',
 'marc245a': 'Lectures on air pollution and environmental impact analyses, Boston, 29 sep.- 3 oct., 1975',
 'marc260a': ['Boston, MA '],
 'marc260c': ['1975 '],
 'marc260b': ['American Meteorological Society'],
 'marc650a': ['Environment Pollution',
  'Air Pollution',
  'Meteorology',
  'Conferences'],
 'marc650completo': ['Environment Pollution',
  'Air Pollution',
  'Meteorology',
  'Conferences'],
 'marc700Coordenador': ['Haugen, D. A. '],
 'marc949a': '71714 ',
 'marc949b': 'DBDSJC ',
 'marc949e': 'Compra ',
 'marc949p': '0 ',
 'marc949y': '25/04/78',
 'marc990a': 'Livro'}

In [ ]:

with open(f'{path}/{listMarcJson[1]}', encoding="utf-8") as file_json:
    s = json.loads(file_json.read())
    file_json.close()
s

CREATE SOLR DOC

In [10]:
path = r'out\goeldi\01-09-22_30-09-22'
listMarcJson = listdir(path)

docs = list()
for i in listMarcJson:
    with open(f'{path}/{i}', encoding="utf-8") as file_json:
        s = json.loads(file_json.read())
        solr_doc = ParserSolr(s, 'Goeldi')
        docs.append(solr_doc)
        print(i)
        file_json.close()

with open(
    f"out/goeldi/solr.json", "w", encoding='utf-8') as file_json:
    json.dump(docs, file_json, ensure_ascii=False, indent=4)
    file_json.close()

35191.json
35192.json
35194.json
35195.json
35197.json
35201.json
35202.json
35203.json
35205.json
35208.json
35221.json
35225.json
35226.json
35227.json
35228.json
35229.json
35297.json
35339.json
35436.json
35450.json
35476.json
35508.json
35510.json
35578.json
35580.json
35581.json
35582.json
35583.json
35584.json
35586.json
35587.json
35588.json
35589.json
35590.json
35591.json
35592.json
35596.json
35597.json
35600.json
35616.json
35618.json
35626.json
